# Preprocessing for complex PDF

## Setting
 - Auto Reload
 - path for utils

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = "../../.."
add_python_path(module_path)

## 1. Create Bedrock client
Amazon Bedrock을 API로 활용하기 위한 Client를 설정합니다. 

In [ ]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [ ]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

## 2. Titan Embedding v2 및 LLM 인 Claude-v3-sonnet 모델 로딩

### LLM 로딩 (Claude-v3-sonnet)

In [ ]:
from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
llm_text = ChatBedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-Sonnet"),
    client=boto3_bedrock,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 2048,
        "stop_sequences": ["\n\nHuman"],
        # "temperature": 0,
        # "top_k": 350,
        # "top_p": 0.999
    }
)
llm_text

### Embedding 모델 선택

In [ ]:
from langchain_aws import BedrockEmbeddings

In [ ]:
llm_emb = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id=bedrock_info.get_model_id(model_name="Titan-Text-Embeddings-V2")
)
dimension = 1024 #1536
print("Bedrock Embeddings Model Loaded")

## 3. 데이터 준비 

### Extract Text, Table and Image from documents

### 3.1 Upstage Document Parse
- ### Prerequisites
    - #### 해당 에셋은 complex document parsing을 위해 [Upstage Document Parse](https://aws.amazon.com/marketplace/pp/prodview-lv5bnpdco7xoq?sr=0-4&ref_=beagle&applicationId=AWSMPContessa)을 이용합니다. 
    - Upstage Document Parse는 AWS Marketplace에 등록되어 있으며, Amazon SageMaker Endpoint 생성을 통해 사용하실 수 있습니다.
        - Endpoint 생성 방식은 **["05_1_create_endpoint_upstage_document_parse.ipynb"](./05_1_create_endpoint_upstage_document_parse.ipynb)** 참고하시기 바랍니다.
- ### **[API Description](https://developers.upstage.ai/docs/apis/document-parse)**


In [ ]:
import shutil
from glob import glob
from requests_toolbelt import MultipartEncoder

import cv2
import math
import base64
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

from utils.common_utils import to_pickle, load_pickle
from langchain.schema import Document

**엔드포인트 이름을 입력합니다.**
- Endpoint 생성 방식은 **["05_1_create_endpoint_upstage_document_parse.ipynb"](./05_1_create_endpoint_upstage_document_parse.ipynb)** 참고하시기 바랍니다.

In [ ]:
endpoint_document_parser = "Upstage-Document-Parse-2024-10-17-21-53-42-914"#"<your endpont>"

**추출할 문서의 위치를 입력합니다.**

In [ ]:
file_path = "./data/complex_pdf/school_edu_guide.pdf"

**추출된 이미지가 저장될 위치를 입력합니다.**

In [ ]:
image_path = "./fig"

**문서 파싱을 시작합니다 (약 1분 소요)**

In [ ]:
%%time

if os.path.isdir(image_path): shutil.rmtree(image_path)
os.mkdir(image_path)

runtime_sm_client = boto3.client('runtime.sagemaker')

# Prepare multipart form data
encoder = MultipartEncoder(
    fields={
        'document': (os.path.basename(file_path), open(file_path, 'rb'), 'application/pdf'),
        'model': 'document-parse',
        'ocr': 'auto',
        'coordinates': 'true',
        'output_formats': '["markdown"]', #'["text", "html", "markdown"]',
        'base64_encoding': '["table", "figure"]'
    }
)

# Get the raw bytes of the multipart form data
body = encoder.to_string()

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_document_parser,
    ContentType=encoder.content_type,  # This will be 'multipart/form-data; boundary=...'
    Body=body
)

result = response["Body"].read()
parse_output = json.loads(result)

In [ ]:
to_pickle(parse_output, "parse_output.pkl")

In [ ]:
parse_output = load_pickle("parse_output.pkl")

**문서 파싱 후처리를 합니다.**
- docs: 문서는 페이지별 정리
- texts: markdown 형식의 전체 texts

In [ ]:
def postprocessing(**kwargs):
    
    category = kwargs["category"]
    markdown = kwargs["markdown"]
    base64_encoding = kwargs["base64_encoding"]
    coordinates = kwargs["coordinates"]
    page = kwargs["page"]
    docs = kwargs["docs"]
    
    if page in docs: 
        docs[page].append({"category": category, "markdown": markdown, "base64_encoding": base64_encoding, "coordinates": coordinates})
    else:
        docs[page] = [{"category": category, "markdown": markdown, "base64_encoding": base64_encoding, "coordinates": coordinates}]
        
    return docs

def extract_image_table(**kwargs):
    
    image_path = kwargs["image_path"]
    file_path = kwargs["file_path"]
    
    image_tmp_path = os.path.join(image_path, "tmp")
    if os.path.isdir(image_tmp_path): shutil.rmtree(image_tmp_path)
    os.mkdir(image_tmp_path)

    # from pdf to image
    pages = convert_from_path(file_path)
    for i, page in enumerate(pages):
        print (f'pdf page {i}, size: {page.size}')    
        page.save(f'{image_tmp_path}/{str(i+1)}.jpg', "JPEG")
        
    return image_tmp_path

In [ ]:
docs = {}
texts = [
    Document(
        page_content=parse_output["content"]["markdown"]
    )
]

## extract_image_table
image_tmp_path = extract_image_table(
    image_path=image_path,
    file_path=file_path
)

for idx, value in enumerate(parse_output["elements"]):
      
    category = value["category"]
    markdown = value["content"]["markdown"]
    page = value["page"]
    
    if category in ["figure", "table"]:
        
        base64_encoding = value["base64_encoding"]    
        coordinates = value["coordinates"]    
        img = Image.open(BytesIO(base64.b64decode(base64_encoding)))
        plt.imshow(img)
        plt.show()
        
        img = cv2.imread(f'{image_tmp_path}/{page}.jpg')
        h, w, c = img.shape
        
        #crob using cv2: img[y: y + h, x: x + w]
        crop_img = img[math.ceil(coordinates[0]["y"]*h):math.ceil(coordinates[3]["y"]*h), \
                       math.ceil(coordinates[0]["x"]*w):math.ceil(coordinates[1]["x"]*w)]
                
        crob_image_path = f'{image_path}/element-{idx}.jpg'
        cv2.imwrite(crob_image_path, crop_img)

        print (f'page_number: {page}')
        print ("==")

        h_crop, w_crop, _ = crop_img.shape
        image_token = w_crop*h_crop/750
        print (f'image: {crob_image_path}, shape: {img.shape}, image_token_for_claude3: {image_token}' )
        
    else:
        base64_encoding= ""
        coordinates=""

    docs = postprocessing(
        docs=docs,
        page=page,
        category=category,
        markdown=markdown,
        base64_encoding=base64_encoding,
        coordinates=coordinates
    )

### Summarization of table and image
- **Image 및 Table 내용을 text 기반으로 요약합니다.**
- **Image 및 Table이 포함된 페이지의 내용을 추가 정보로 사용하여 문맥을 고려한 요약을 합니다.**
- **이 과정은 image 및 table 정보를 embedding 기반 모델로 검색 가능하게 합니다.**
- BedrockChat with claude3: https://medium.com/@dminhk/building-with-anthropics-claude-3-on-amazon-bedrock-and-langchain-%EF%B8%8F-2b842f9c0ca8

In [ ]:
import copy
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

**요약 대상과 그에 대한 컨텍스트 정보를 구분합니다**

In [ ]:
docs_for_summary = []
for page, elements in docs.items():
    
    elements = [element for element in elements if element["category"] != "footer"]
    print (f'page: {page}, # elements: {len(elements)}')
    
    for idx, element in enumerate(elements):
        
        category, markdown = element["category"], element["markdown"]
        print (f'element idx: {idx}, category: {element["category"]}')
        
        elements_copy = copy.deepcopy(elements)
        if category in ("figure", "table"):  
            
            summary_target = elements_copy.pop(idx)
            contexts_markdown = '\n'.join([context["markdown"] for context in elements_copy])
            docs_for_summary.append(
                {
                    "target_category": summary_target["category"],
                    "target_base64": summary_target["base64_encoding"],
                    "target_markdown": summary_target["markdown"],
                    "contexts_markdown": contexts_markdown
                }
            )

**정보를 요약합니다**

In [ ]:
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import botocore
from utils.common_utils import retry

In [ ]:
system_prompt = "You are an assistant tasked with describing table and image."
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": "data:image/png;base64," + "{image_base64}",
        },
    },
    {
        "type": "text",
        "text": '''
                 <contexts>
                 {contexts}
                 </contexts>
                 
                 주어진 이미지 또는 테이블을 자세히 분석하고 주어진 contexts를 참고하여 다음 정보를 추출해주세요:
                 
                 1. <title> 태그 안의 제목을 정확히 제시해주세요.
                 2. <summary> 태그 안의 내용을 요약해주세요.
                 3. <entities> 태그 안의 모든 항목을 나열하고, 각 항목에 대한 간단한 설명을 제공해주세요.
                 4. <hypothetical_questions> 태그 안의 질문들을 모두 나열해주세요.
                모든 정보는 원본 내용을 정확히 반영하되, 필요한 경우 약간의 추가 설명을 덧붙여 이해를 돕도록 해주세요.
        '''
    },
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        system_message_template,
        human_message_template
    ]
)

summarize_chain = prompt | llm_text | StrOutputParser()

In [ ]:
@retry(total_try_cnt=5, sleep_in_sec=10, retryable_exceptions=(botocore.exceptions.EventStreamError))
def summary_img(summarize_chain, image_base64, contexts):

    img = Image.open(BytesIO(base64.b64decode(image_base64)))
    plt.imshow(img)
    plt.show()

    stream = summarize_chain.stream(
        {
            "image_base64": image_base64,
            "contexts": contexts
        }
    )
    response = ""
    for chunk in stream: response += chunk

    return response

In [ ]:
summaries = []
for idx, doc in enumerate(docs_for_summary):
    summary = summary_img(summarize_chain, doc["target_base64"], doc["contexts_markdown"])
    summaries.append(summary)
    print ("\n==")
    print (idx)

In [ ]:
verbose = True
if verbose:
    for doc, summary in zip(docs_for_summary, summaries):

        print ("============================")
        img = Image.open(BytesIO(base64.b64decode(doc["target_base64"])))
        plt.imshow(img)
        plt.show()

        print (f'Summary: {summary}')

`요약`된 내용을 Document의 `page_content`로, `OCR`결과는 metadata의 `origin_image`로 사용

In [ ]:
images_preprocessed = []

for doc, summary in zip(docs_for_summary, summaries):
    
    metadata = {}
    metadata["markdown"] = doc["target_markdown"]
    metadata["category"] = "Image"
    metadata["image_base64"] = doc["target_base64"]
    
    doc = Document(
        page_content=summary,
        metadata=metadata
    )
    images_preprocessed.append(doc)

In [ ]:
images_preprocessed[0]

### For tables (summary for markdown)

In [ ]:
tables = [doc for doc in docs_for_summary if doc["target_category"] == "table"]

In [ ]:
human_prompt = [
    {
        "type": "text",
        "text": '''
                 Here is the table: <table>{table}</table>
                 Given table, give a concise summary.
                 Don't insert any XML tag such as <table> and </table> when answering.
                 Write in Korean.
        '''
    },
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        system_message_template,
        human_message_template
    ]
)

#summarize_chain = prompt | llm_text | StrOutputParser()
summarize_chain = {"table": lambda x:x} | prompt | llm_text | StrOutputParser()

In [ ]:
table_info = [t["target_markdown"] for t in tables]
table_summaries = summarize_chain.batch(table_info, config={"max_concurrency": 1})

`요약`된 내용을 Document의 `page_content`로, `parsed table`은 metadata의 `origin_table`로 사용

In [ ]:
tables_preprocessed = []

for doc, summary in zip(tables, table_summaries):
    
    metadata = {}
    metadata["origin_table"] = doc["target_markdown"]
    metadata["text_as_html"] = doc["target_markdown"]
    metadata["category"] = "Table"
    metadata["image_base64"] = doc["target_base64"]
    
    doc = Document(
        page_content=summary,
        metadata=metadata
    )
    tables_preprocessed.append(doc)

In [ ]:
tables_preprocessed

In [ ]:
verbose = True
index = 0
if verbose:
    for table, summary in zip(tables, table_summaries):

        #if table_as_image:
        #page_contents, table_as_html, img_base64 = table
        category, img_base64, table_as_html, page_contents = table["target_category"], table["target_base64"], table["target_markdown"], table["target_markdown"]
        #else: page_contents, table_as_html = table

        print ("============================")
        print (index)
        print (f'table: {page_contents}')
        print ("----------------------------")
        print (f'summary: {summary}')
        print ("----------------------------")

        print ("----------------------------")
        if img_base64 is not None:
            print ("image")
            img = Image.open(BytesIO(base64.b64decode(img_base64)))
            plt.imshow(img)
            plt.show()
        index += 1


## 4. Index 생성
Document로 부터 추출한 text, table, 그리고 image 정보를 Amazon OpenSearch에 저장합니다. (인덱싱)  

### Index 이름 결정

In [ ]:
import boto3
from utils.ssm import parameter_store

In [ ]:
region=boto3.Session().region_name
pm = parameter_store(region)

In [ ]:
index_name = "complex-doc-index"

In [ ]:
pm.put_params(
    key="opensearch_index_name",
    value=f'{index_name}',
    overwrite=True,
    enc=False
)

### Index 스키마 정의

In [ ]:
index_body = {
    'settings': {
        'analysis': {
            'analyzer': {
                'my_analyzer': {
                         'char_filter':['html_strip'],
                    'tokenizer': 'nori',
                    'filter': [
                        #'nori_number',
                        #'lowercase',
                        #'trim',
                        'my_nori_part_of_speech'
                    ],
                    'type': 'custom'
                }
            },
            'tokenizer': {
                'nori': {
                    'decompound_mode': 'mixed',
                    'discard_punctuation': 'true',
                    'type': 'nori_tokenizer'
                }
            },
            "filter": {
                "my_nori_part_of_speech": {
                    "type": "nori_part_of_speech",
                    "stoptags": [
                        "J", "XSV", "E", "IC","MAJ","NNB",
                        "SP", "SSC", "SSO",
                        "SC","SE","XSN","XSV",
                        "UNA","NA","VCP","VSV",
                        "VX"
                    ]
                }
            }
        },
        'index': {
            'knn': True,
            'knn.space_type': 'cosinesimil'  # Example space type
        }
    },
    'mappings': {
        'properties': {
            'metadata': {
                'properties': {
                    'source': {'type': 'keyword'},
                    'page_number': {'type':'long'},
                    'category': {'type':'text'},
                    'file_directory': {'type':'text'},
                    'last_modified': {'type': 'text'},
                    'type': {'type': 'keyword'},
                    'image_base64': {'type':'text'},
                    'origin_image': {'type':'text'},
                    'origin_table': {'type':'text'},
                }
            },
            'text': {
                'analyzer': 'my_analyzer',
                'search_analyzer': 'my_analyzer',
                'type': 'text'
            },
            'vector_field': {
                'type': 'knn_vector',
                'dimension': f"{dimension}" # Replace with your vector dimension
            }
        }
    }
}


## 5. LangChain OpenSearch VectorStore 생성 
### 선수 조건

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.

In [ ]:
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)

opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)

In [ ]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### OpenSearch Client 생성

In [ ]:
from utils.opensearch import opensearch_utils

In [ ]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

### 오픈 서치 인덱스 생성 
- 오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [ ]:
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )

opensearch_utils.create_index(os_client, index_name, index_body)
index_info = os_client.indices.get(index=index_name)
print("Index is created")
pprint(index_info)

### 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [ ]:
#from langchain.vectorstores import OpenSearchVectorSearch
from langchain_community.vectorstores import OpenSearchVectorSearch

In [ ]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss=False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60
)
vector_db

### Chunking Strategy: Parent documents
* Chunk Size and Chunk Overlap Size

In [ ]:
from utils.chunk import parant_documents

In [ ]:
parent_chunk_size = 4096
parent_chunk_overlap = 0

child_chunk_size = 1024
child_chunk_overlap = 256

opensearch_parent_key_name = "parent_id"
opensearch_family_tree_key_name = "family_tree"

* Parent Chunking
    - create_parent_chunk() 아래와 같은 작업을 합니다.
    - all_docs 에 있는 문서를 parent_chunk_size 만큼으로 청킹 합니다.
    - Parent Chunk 에 두개의 메타 데이타를 생성 합니다.
        - family_tree: parent
        - parent_id : None

In [ ]:
parent_chunk_docs = parant_documents.create_parent_chunk(
    docs=texts,
    parent_id_key=opensearch_parent_key_name,
    family_tree_id_key=opensearch_family_tree_key_name,
    parent_chunk_size=parent_chunk_size,
    parent_chunk_overlap=parent_chunk_overlap
)
print(f'Number of parent_chunk_docs= {len(parent_chunk_docs)}')

* Insert doc into OpenSearch 

In [ ]:
%%time

parent_ids = vector_db.add_documents(
    documents = parent_chunk_docs, 
    vector_field = "vector_field",
    bulk_size = 1000000
)

In [ ]:
total_count_docs = opensearch_utils.get_count(os_client, index_name)
print("total count docs: ", total_count_docs)

* 삽입된 Parent Chunk 의 첫번째를 확인 합니다. family_tree, parent_id 의 값을 확인 하세요.

In [ ]:
def show_opensearch_doc_info(response):
    print("opensearch document id:" , response["_id"])
    print("family_tree:" , response["_source"]["metadata"]["family_tree"])
    print("parent document id:" , response["_source"]["metadata"]["parent_id"])
    print("parent document text: \n" , response["_source"]["text"])

response = opensearch_utils.get_document(os_client, doc_id = parent_ids[0], index_name = index_name)
show_opensearch_doc_info(response)

* Child Chunking
    - 아래의 create_child_chunk() 는 다음과 같은 작업을 합니다.
    - parent_chunk_docs 각각에 대해서 Child Chunk 를 생성 합니다. 
    - Child Chunk 에 두개의 메타 데이타를 생성 합니다.
        - family_tree: child
        - parent_id : parent 에 대한 OpenSearch document id

In [ ]:
# child_chunk_docs = create_child_chunk(parent_chunk_docs[0:1], parent_ids)
child_chunk_docs = parant_documents.create_child_chunk(
    child_chunk_size=child_chunk_size,
    child_chunk_overlap=child_chunk_overlap,
    docs=parent_chunk_docs,
    parent_ids_value=parent_ids,
    parent_id_key=opensearch_parent_key_name,
    family_tree_id_key=opensearch_family_tree_key_name
)

print(f"Number of child_chunk_docs= {len(child_chunk_docs)}")

In [ ]:
parent_id = child_chunk_docs[0].metadata["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = index_name)
show_opensearch_doc_info(response)    

In [ ]:
child_chunk_docs[0]

### Manipulate table and image for parent documents
- set family_tree of table and image documents as "parent_table" and these kinds of types are included in searcable list (child, parent_table and parent_image) in parant document strategy.

In [ ]:
for table in tables_preprocessed:
    table.metadata["family_tree"], table.metadata["parent_id"] = "parent_table", "NA"

In [ ]:
for image in images_preprocessed:
    image.metadata["family_tree"], image.metadata["parent_id"] = "parent_image", "NA"

### Merge text, table and image

In [ ]:
from itertools import chain

In [ ]:
docs_preprocessed = list(chain(child_chunk_docs, tables_preprocessed, images_preprocessed))

* Insert doc into OpenSearch

In [ ]:
%%time

child_ids = vector_db.add_documents(
    documents=docs_preprocessed, 
    vector_field = "vector_field",
    bulk_size=1000000
)

print("length of child_ids: ", len(child_ids))

## 6. 검색 테스트

In [ ]:
from utils.rag import qa_chain
from utils.rag import prompt_repo, show_context_used
from langchain.callbacks.tracers import ConsoleCallbackHandler
from utils.rag import retriever_utils, OpenSearchHybridSearchRetriever

In [ ]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    index_name=index_name,
    llm_text=llm_text, # llm for query augmentation in both rag_fusion and HyDE
    llm_emb=llm_emb, # Used in semantic search based on opensearch 

    # hybird-search debugger
    #hybrid_search_debugger = "semantic", #[semantic, lexical, None]
    
    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for search
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.51, .49], # [for semantic, for lexical], Semantic, Lexical search 결과에 대한 최종 반영 비율 정의
    reranker=False, # enable reranker with reranker model
    #reranker_endpoint_name=endpoint_name, # endpoint name for reranking model
    parent_document=True, # enable parent document
    
    # option for complex pdf consisting of text, table and image
    complex_doc=True,
    
    # option for async search
    async_mode=True,

    # option for output
    k=7, # 최종 Document 수 정의
    verbose=False,

)

### 하이브리드 검색

In [ ]:
%%time
#query = "자동조기 상환"
query = "디지털 성범죄"
search_hybrid_result, tables, images = opensearch_hybrid_retriever.invoke(query)

In [ ]:
show_context_used(search_hybrid_result)

# SageMaker Processing Job 이용하기

## 1. scripy 만들기

In [ ]:
%%writefile ./src/preprocessing.py

import os
import json
import copy
import boto3
import shutil
import argparse
from pprint import pprint
from utils import bedrock
from itertools import chain
from utils.bedrock import bedrock_info

from langchain_aws import ChatBedrock
from langchain_aws import BedrockEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

import cv2
import math
import base64
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from langchain.schema import Document
from pdf2image import convert_from_path
from requests_toolbelt import MultipartEncoder

import botocore
from utils.common_utils import retry
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

from utils.chunk import parant_documents
from utils.opensearch import opensearch_utils
from langchain_community.vectorstores import OpenSearchVectorSearch

class preprocess():
    
    def __init__(self, args):
        
        self.args = args
        self.proc_prefix = self.args.proc_prefix #'/opt/ml/processing'
        
        self.input_dir = os.path.join(self.proc_prefix, "input")
        self.output_dir = os.path.join(self.proc_prefix, "output")
        os.makedirs(self.input_dir, exist_ok=True)
        os.makedirs(self.output_dir, exist_ok=True)
        
        print (os.listdir(self.input_dir))
        
        ############# 수정
        #self.file_path = os.path.join(self.input_dir, "sample-2.pdf")
        self.file_path = os.path.join(self.input_dir, self.args.file_name)
        #"./data/complex_pdf/sample-2.pdf"
        ###############
        
    def _initialization(self, ):
        
        self.boto3_bedrock = bedrock.get_bedrock_client(
            assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
            endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
            region=os.environ.get("AWS_DEFAULT_REGION", None),
        )
        
        self.llm_text = ChatBedrock(
            model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-Sonnet"),
            client=self.boto3_bedrock,
            streaming=True,
            callbacks=[StreamingStdOutCallbackHandler()],
            model_kwargs={
                "max_tokens": 2048,
                "stop_sequences": ["\n\nHuman"],
            }
        )
        
        self.llm_emb = BedrockEmbeddings(
            client=self.boto3_bedrock,
            model_id=bedrock_info.get_model_id(model_name="Titan-Text-Embeddings-V2")
        )
        self.dimension = 1024
        
    def _document_parsing(self, ):
        
        runtime_sm_client = boto3.client('runtime.sagemaker')

        # Prepare multipart form data
        encoder = MultipartEncoder(
            fields={
                'document': (os.path.basename(self.file_path), open(self.file_path, 'rb'), 'application/pdf'),
                'model': 'document-parse',
                'ocr': 'auto',
                'coordinates': 'true',
                'output_formats': '["markdown"]', #'["text", "html", "markdown"]',
                'base64_encoding': '["table", "figure"]'
            }
        )

        # Get the raw bytes of the multipart form data
        body = encoder.to_string()

        response = runtime_sm_client.invoke_endpoint(
            EndpointName=self.args.endpoint_document_parser,
            ContentType=encoder.content_type,  # This will be 'multipart/form-data; boundary=...'
            Body=body
        )

        result = response["Body"].read()
        parse_output = json.loads(result)
        
        return parse_output
    
    def _extract_image_table(self, parse_output):
        
        def processing(**kwargs):
    
            category = kwargs["category"]
            markdown = kwargs["markdown"]
            base64_encoding = kwargs["base64_encoding"]
            coordinates = kwargs["coordinates"]
            page = kwargs["page"]
            docs = kwargs["docs"]

            if page in docs: 
                docs[page].append(
                    {
                        "category": category,
                         "markdown": markdown,
                         "base64_encoding": base64_encoding,
                         "coordinates": coordinates
                    }
                )
            else:
                docs[page] = [
                    {
                        "category": category,
                        "markdown": markdown,
                        "base64_encoding": base64_encoding,
                        "coordinates": coordinates
                    }
                ]

            return docs

        def image_conversion(**kwargs):

            image_path = kwargs["image_path"]
            file_path = kwargs["file_path"]

            image_tmp_path = os.path.join(image_path, "tmp")
            if os.path.isdir(image_tmp_path): shutil.rmtree(image_tmp_path)
            os.mkdir(image_tmp_path)

            # from pdf to image
            pages = convert_from_path(file_path)
            for i, page in enumerate(pages):
                print (f'pdf page {i}, size: {page.size}')    
                page.save(f'{image_tmp_path}/{str(i+1)}.jpg', "JPEG")

            return image_tmp_path
        
        image_path = os.path.join(self.input_dir, "fig")
        if os.path.isdir(image_path): shutil.rmtree(image_path)
        os.mkdir(image_path)
        
        docs = {}
        texts = [
            Document(
                page_content=parse_output["content"]["markdown"]
            )
        ]

        ## extract_image_table
        image_tmp_path = image_conversion(
            image_path=image_path,
            file_path=self.file_path
        )

        for idx, value in enumerate(parse_output["elements"]):

            category = value["category"]
            markdown = value["content"]["markdown"]
            page = value["page"]

            if category in ["figure", "table"]:

                base64_encoding = value["base64_encoding"]    
                coordinates = value["coordinates"]    
                img = Image.open(BytesIO(base64.b64decode(base64_encoding)))
                plt.imshow(img)
                plt.show()

                img = cv2.imread(f'{image_tmp_path}/{page}.jpg')
                h, w, c = img.shape

                #crob using cv2: img[y: y + h, x: x + w]
                crop_img = img[math.ceil(coordinates[0]["y"]*h):math.ceil(coordinates[3]["y"]*h), \
                               math.ceil(coordinates[0]["x"]*w):math.ceil(coordinates[1]["x"]*w)]

                crob_image_path = f'{image_path}/element-{idx}.jpg'
                cv2.imwrite(crob_image_path, crop_img)

                print (f'page_number: {page}')
                print ("==")

                h_crop, w_crop, _ = crop_img.shape
                image_token = w_crop*h_crop/750
                print (f'image: {crob_image_path}, shape: {img.shape}, image_token_for_claude3: {image_token}' )

            else:
                base64_encoding= ""
                coordinates=""

            docs = processing(
                docs=docs,
                page=page,
                category=category,
                markdown=markdown,
                base64_encoding=base64_encoding,
                coordinates=coordinates
            )
            
        return docs, texts
    
    def _context_generation_for_image(self, docs):
        
        def manipulate_docs_for_summary(docs):
            
            docs_for_summary = []
            for page, elements in docs.items():

                elements = [element for element in elements if element["category"] != "footer"]
                print (f'page: {page}, # elements: {len(elements)}')

                for idx, element in enumerate(elements):

                    category, markdown = element["category"], element["markdown"]
                    print (f'element idx: {idx}, category: {element["category"]}')

                    elements_copy = copy.deepcopy(elements)
                    if category in ("figure", "table"):  
                        
                        summary_target = elements_copy.pop(idx)
                        contexts_markdown = '\n'.join([context["markdown"] for context in elements_copy])
                        docs_for_summary.append(
                            {
                                "target_category": summary_target["category"],
                                "target_base64": summary_target["base64_encoding"],
                                "target_markdown": summary_target["markdown"],
                                "contexts_markdown": contexts_markdown
                            }
                        )
                
            return docs_for_summary
        
        def get_summary_chain():
            
            system_prompt = "You are an assistant tasked with describing table and image."
            system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

            human_prompt = [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "data:image/png;base64," + "{image_base64}",
                    },
                },
                {
                    "type": "text",
                    "text": '''
                             <contexts>
                             {contexts}
                             </contexts>

                             주어진 이미지 또는 테이블을 자세히 분석하고 주어진 contexts를 참고하여 다음 정보를 추출해주세요:

                             1. <title> 태그 안의 제목을 정확히 제시해주세요.
                             2. <summary> 태그 안의 내용을 요약해주세요.
                             3. <entities> 태그 안의 모든 항목을 나열하고, 각 항목에 대한 간단한 설명을 제공해주세요.
                             4. <hypothetical_questions> 태그 안의 질문들을 모두 나열해주세요.
                            모든 정보는 원본 내용을 정확히 반영하되, 필요한 경우 약간의 추가 설명을 덧붙여 이해를 돕도록 해주세요.
                    '''
                },
            ]
            human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)
            
            prompt = ChatPromptTemplate.from_messages(
                [
                    system_message_template,
                    human_message_template
                ]
            )

            summarize_chain = prompt | self.llm_text | StrOutputParser()
            
            return summarize_chain
        
        @retry(total_try_cnt=5, sleep_in_sec=10, retryable_exceptions=(botocore.exceptions.EventStreamError))
        def summary_img(summarize_chain, image_base64, contexts):

            img = Image.open(BytesIO(base64.b64decode(image_base64)))
            plt.imshow(img)
            plt.show()

            stream = summarize_chain.stream(
                {
                    "image_base64": image_base64,
                    "contexts": contexts
                }
            )
            response = ""
            for chunk in stream: response += chunk

            return response
            
        docs_for_summary = manipulate_docs_for_summary(docs)
        
        print ("docs_for_summary", len(docs_for_summary))
        
        
        summarize_chain = get_summary_chain()
        
        summaries = []
        for idx, doc in enumerate(docs_for_summary):
            summary = summary_img(summarize_chain, doc["target_base64"], doc["contexts_markdown"])
            summaries.append(summary)
            print ("\n==")
            print (idx)
            
        images_preprocessed = []
        for doc, summary in zip(docs_for_summary, summaries):

            metadata = {}
            metadata["markdown"] = doc["target_markdown"]
            metadata["category"] = "Image"
            metadata["image_base64"] = doc["target_base64"]

            doc = Document(
                page_content=summary,
                metadata=metadata
            )
            images_preprocessed.append(doc)
            
        for image in images_preprocessed:
            image.metadata["family_tree"], image.metadata["parent_id"] = "parent_image", "NA"
        
        return images_preprocessed, docs_for_summary

    def _context_generation_for_table(self, docs_for_summary):
        
        def get_summary_chain():
            
            system_prompt = "You are an assistant tasked with describing table and image."
            system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)
            
            human_prompt = [
                {
                    "type": "text",
                    "text": '''
                             Here is the table: <table>{table}</table>
                             Given table, give a concise summary.
                             Don't insert any XML tag such as <table> and </table> when answering.
                             Write in Korean.
                    '''
                },
            ]
            human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

            prompt = ChatPromptTemplate.from_messages(
                [
                    system_message_template,
                    human_message_template
                ]
            )

            summarize_chain = {"table": lambda x:x} | prompt | self.llm_text | StrOutputParser()
            
            return summarize_chain
        
        tables = [doc for doc in docs_for_summary if doc["target_category"] == "table"]
        summarize_chain = get_summary_chain()
        table_info = [t["target_markdown"] for t in tables]
        table_summaries = summarize_chain.batch(table_info, config={"max_concurrency": 1})
        
        tables_preprocessed = []
        for doc, summary in zip(tables, table_summaries):

            metadata = {}
            metadata["origin_table"] = doc["target_markdown"]
            metadata["text_as_html"] = doc["target_markdown"]
            metadata["category"] = "Table"
            metadata["image_base64"] = doc["target_base64"]

            doc = Document(
                page_content=summary,
                metadata=metadata
            )
            tables_preprocessed.append(doc)
        
        for table in tables_preprocessed:
            table.metadata["family_tree"], table.metadata["parent_id"] = "parent_table", "NA"
            
        return tables_preprocessed
    
    def _opensearch(self, ):
        
        index_body = {
            'settings': {
                'analysis': {
                    'analyzer': {
                        'my_analyzer': {
                                 'char_filter':['html_strip'],
                            'tokenizer': 'nori',
                            'filter': [
                                #'nori_number',
                                #'lowercase',
                                #'trim',
                                'my_nori_part_of_speech'
                            ],
                            'type': 'custom'
                        }
                    },
                    'tokenizer': {
                        'nori': {
                            'decompound_mode': 'mixed',
                            'discard_punctuation': 'true',
                            'type': 'nori_tokenizer'
                        }
                    },
                    "filter": {
                        "my_nori_part_of_speech": {
                            "type": "nori_part_of_speech",
                            "stoptags": [
                                "J", "XSV", "E", "IC","MAJ","NNB",
                                "SP", "SSC", "SSO",
                                "SC","SE","XSN","XSV",
                                "UNA","NA","VCP","VSV",
                                "VX"
                            ]
                        }
                    }
                },
                'index': {
                    'knn': True,
                    'knn.space_type': 'cosinesimil'  # Example space type
                }
            },
            'mappings': {
                'properties': {
                    'metadata': {
                        'properties': {
                            'source': {'type': 'keyword'},
                            'page_number': {'type':'long'},
                            'category': {'type':'text'},
                            'file_directory': {'type':'text'},
                            'last_modified': {'type': 'text'},
                            'type': {'type': 'keyword'},
                            'image_base64': {'type':'text'},
                            'origin_image': {'type':'text'},
                            'origin_table': {'type':'text'},
                        }
                    },
                    'text': {
                        'analyzer': 'my_analyzer',
                        'search_analyzer': 'my_analyzer',
                        'type': 'text'
                    },
                    'vector_field': {
                        'type': 'knn_vector',
                        'dimension': f"{self.dimension}" # Replace with your vector dimension
                    }
                }
            }
        }

        opensearch_domain_endpoint = self.args.opensearch_domain_endpoint
        opensearch_user_id = self.args.opensearch_user_id
        opensearch_user_password = self.args.opensearch_user_password
        index_name = self.args.index_name
        
        http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password
        
        ## opensearch clinet 생성
        aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
        os_client = opensearch_utils.create_aws_opensearch_client(
            aws_region,
            opensearch_domain_endpoint,
            http_auth
        )
        
        ## opensearch index 생성
        index_exists = opensearch_utils.check_if_index_exists(
            os_client,
            index_name
        )

        if index_exists:
            opensearch_utils.delete_index(
                os_client,
                index_name
            )

        opensearch_utils.create_index(os_client, index_name, index_body)
        index_info = os_client.indices.get(index=index_name)
        print("Index is created")
        pprint(index_info)
        
        vector_db = OpenSearchVectorSearch(
            index_name=index_name,
            opensearch_url=opensearch_domain_endpoint,
            embedding_function=self.llm_emb,
            http_auth=http_auth, # http_auth
            is_aoss=False,
            engine="faiss",
            space_type="l2",
            bulk_size=100000,
            timeout=60
        )
        
        return os_client, vector_db, index_name
        
    def _chunking_and_indexing(self, os_client, vector_db, index_name, texts, images_preprocessed, tables_preprocessed):
        
        parent_chunk_size = 4096
        parent_chunk_overlap = 0

        child_chunk_size = 1024
        child_chunk_overlap = 256

        opensearch_parent_key_name = "parent_id"
        opensearch_family_tree_key_name = "family_tree"
        
        parent_chunk_docs = parant_documents.create_parent_chunk(
            docs=texts,
            parent_id_key=opensearch_parent_key_name,
            family_tree_id_key=opensearch_family_tree_key_name,
            parent_chunk_size=parent_chunk_size,
            parent_chunk_overlap=parent_chunk_overlap
        )
        print(f'Number of parent_chunk_docs= {len(parent_chunk_docs)}')
        parent_ids = vector_db.add_documents(
            documents = parent_chunk_docs, 
            vector_field = "vector_field",
            bulk_size = 1000000
        )
        
        total_count_docs = opensearch_utils.get_count(os_client, index_name)
        print("total count docs: ", total_count_docs)
                
        # child_chunk_docs = create_child_chunk(parent_chunk_docs[0:1], parent_ids)
        child_chunk_docs = parant_documents.create_child_chunk(
            child_chunk_size=child_chunk_size,
            child_chunk_overlap=child_chunk_overlap,
            docs=parent_chunk_docs,
            parent_ids_value=parent_ids,
            parent_id_key=opensearch_parent_key_name,
            family_tree_id_key=opensearch_family_tree_key_name
        )

        print(f"Number of child_chunk_docs= {len(child_chunk_docs)}")
        parent_id = child_chunk_docs[0].metadata["parent_id"]
        print("child's parent_id: ", parent_id)
        print("\n###### Search parent in OpenSearch")
        
        
        ## Merge
        docs_preprocessed = list(chain(child_chunk_docs, tables_preprocessed, images_preprocessed))
        
        child_ids = vector_db.add_documents(
            documents=docs_preprocessed, 
            vector_field = "vector_field",
            bulk_size=1000000
        )
        print("length of child_ids: ", len(child_ids))
        
        return child_chunk_docs
        
    def execution(self, ):
        
        ## Initialization for Bedrock
        self._initialization()
        
        ## Document parsing
        parse_output = self._document_parsing()
        
        ## Context generation for images and tables
        docs, texts = self._extract_image_table(parse_output)
        images_preprocessed, docs_for_summary = self._context_generation_for_image(docs)
        tables_preprocessed = self._context_generation_for_table(docs_for_summary)
        
        ## Opensearch setting
        os_client, vector_db, index_name = self._opensearch()
        
        ## Chunking(Parent document) and indexing
        child_chunk_docs = self._chunking_and_indexing(os_client, vector_db, index_name, texts, images_preprocessed, tables_preprocessed)
        
if __name__ == "__main__":
    
    parser = argparse.ArgumentParser()
    parser.add_argument("--region", type=str, default="us-west-2")
    parser.add_argument("--proc_prefix", type=str, default="./src")
    parser.add_argument("--endpoint_document_parser", type=str, default="")
    parser.add_argument("--opensearch_domain_endpoint", type=str, default="")
    parser.add_argument("--opensearch_user_id", type=str, default="")
    parser.add_argument("--opensearch_user_password", type=str, default="")
    parser.add_argument("--index_name", type=str, default="")
    parser.add_argument("--file_name", type=str, default="")
    
    args, _ = parser.parse_known_args()

    print("Received arguments {}".format(args))
    
    os.environ['AWS_DEFAULT_REGION'] = args.region
        
    prep = preprocess(args)
    prep.execution()
    

In [ ]:
#!python ./src/preprocessing.py

## 2. parser용 이미지 생성하기
 **[중요] ECR 사용을 위해 "AmazonEC2ContainerRegistryFullAccess" policy를 role에 추가해야 합니다**

In [ ]:
import sagemaker
from utils.ecr import ecr_handler

In [ ]:
ecr = ecr_handler()
region=boto3.Session().region_name
strAccountID = boto3.client("sts").get_caller_identity().get("Account")

In [ ]:
container_uri = sagemaker.image_uris.retrieve(
    framework="sklearn", version="1.2-1", region=region
)
container_uri

In [ ]:
%%writefile ./custom-docker/Dockerfile-docparse

FROM 246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3

RUN wget -O - https://apt.kitware.com/keys/kitware-archive-latest.asc 2>/dev/null | gpg --dearmor - | tee /usr/share/keyrings/kitware-archive-keyring.gpg >/dev/null
RUN apt-get update
RUN apt-get install -y libgl1-mesa-glx
RUN apt-get install -y libglib2.0-0
RUN apt-get install -y poppler-utils

RUN pip install -U pip
RUN pip install botocore==1.35.40
RUN pip install boto3==1.35.40
RUN pip install sagemaker==2.232.2
RUN pip install langchain==0.3.3
RUN pip install langchain_aws==0.2.2
RUN pip install langchain_community==0.3.2
RUN pip install requests_toolbelt==1.0.0
RUN pip install pdf2image==1.17.0
RUN pip install numpy==1.26.4
RUN pip install opencv-python==4.10.0.84
RUN pip install ipython==8.18.1
RUN pip install opensearch-py==2.7.1

ENV PYTHONUNBUFFERED=TRUE

In [ ]:
strRepositoryName="docparse-image"  ## <-- 원하는 docker repostory 이름을 추가
strRepositoryName = strRepositoryName.lower()
strDockerFile = "Dockerfile-docparse"
strDockerDir = "./custom-docker/"
strTag = "latest"

In [ ]:
## Build
#ecr.build_docker(strDockerDir, strDockerFile, strRepositoryName, strRegionName="ap-northeast-2", strAccountId="246618743249")
ecr.build_docker(strDockerDir, strDockerFile, strRepositoryName, strRegionName="us-west-2", strAccountId="246618743249")

In [ ]:
## Upload to ECR
strEcrRepositoryUri = ecr.register_image_to_ecr(region, strAccountID, strRepositoryName, strTag)

## 3. 파일 업로드 to S3

In [ ]:
import sagemaker
from utils.s3 import s3_handler

In [ ]:
sagemaker_session = sagemaker.Session()
strBucketName = sagemaker_session.default_bucket() # 기본 버킷 이름 가져오기
strLocalDataPath = "./data/complex_pdf/"

In [ ]:
s3 = s3_handler()
source_dir, target_bucket, target_dir = strLocalDataPath, strBucketName, "doc_parse_data"
s3.upload_dir(source_dir, target_bucket, target_dir)

strS3DataPath = f's3://{strBucketName}/doc_parse_data'

## 3. Processing Job 실행

In [ ]:
import os
from utils.ssm import parameter_store
from sagemaker import get_execution_role
#from sagemaker.pytorch.estimator import PyTorch
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.processing import ProcessingInput, ProcessingOutput, FrameworkProcessor

In [ ]:
strRegionName = boto3.Session().region_name
strAccountId = boto3.client("sts").get_caller_identity().get("Account")
strPrefix = "doc-parse"

In [ ]:
local_mode = False

strExecutionRole = get_execution_role()

if local_mode: 
    strInstanceType = 'local'
    
    import os
    from sagemaker.local import LocalSession
    
    sagemaker_session = LocalSession()
    strDataPath = str(os.path.join("file://", os.getcwd(), "data", "complex_pdf"))
    
else:
    strInstanceType = "ml.m5.xlarge"
    sagemaker_session = sagemaker.Session()
    strDataPath = strS3DataPath
        
print (f"instance-type: {strInstanceType}")
print (f'role: {strExecutionRole}')
print (f"bucket: {strBucketName}")
print (f"dataset-path: {strDataPath}")
print (f"sagemaker_session: {sagemaker_session}")

In [ ]:
dataset_processor = FrameworkProcessor(
    estimator_cls=SKLearn,
    framework_version="1.2-1",
    py_version="py3",
    image_uri="615299776985.dkr.ecr.us-west-2.amazonaws.com/docparse-image",#"<your image uri>"
    instance_type=strInstanceType,
    instance_count=1,
    role=strExecutionRole,
    base_job_name="preprocessing", # bucket에 보이는 이름 (pipeline으로 묶으면 pipeline에서 정의한 이름으로 bucket에 보임)
    sagemaker_session=sagemaker_session
)

strProcPrefix = "/opt/ml/processing"
strOutputPath = os.path.join(
    "s3://{}".format(strBucketName),
    strPrefix,
    "preprocessing",
    "output"
)

pm = parameter_store(strRegionName)
endpoint_document_parser = "Upstage-Document-Parse-2024-10-17-21-53-42-914"
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)
opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)
opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)
index_name = "sm_prep_job"
file_name = "sample-2.pdf"

In [ ]:
dataset_processor.run(
    #job_name="preprocessing", ## 이걸 넣어야 캐시가 작동함, 안그러면 프로세서의 base_job_name 이름뒤에 날짜 시간이 붙어서 캐시 동작 안함
    #git_config=git_config,
    code='preprocessing.py', #소스 디렉토리 안에서 파일 path
    source_dir= "./src", #현재 파일에서 소스 디렉토리 상대경로 # add processing.py and requirements.txt here
    inputs=[
        ProcessingInput(
            input_name="input-data",
            source=strDataPath,
            destination=os.path.join(strProcPrefix, "input")
        ),
    ],
    outputs=[       
        ProcessingOutput(
            output_name="output-data",
            source=os.path.join(strProcPrefix, "output"),
            destination=strOutputPath
        ),
    ],
    arguments=[
        "--proc_prefix", strProcPrefix, \
        "--region", region, \
        "--endpoint_document_parser", endpoint_document_parser, \
        "--opensearch_domain_endpoint", opensearch_domain_endpoint, \
        "--opensearch_user_id", opensearch_user_id, \
        "--opensearch_user_password", opensearch_user_password, \
        "--index_name", index_name, \
        "--file_name", file_name
    ]
)


## 확인하기

In [ ]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    index_name="sm_prep_job",
    llm_text=llm_text, # llm for query augmentation in both rag_fusion and HyDE
    llm_emb=llm_emb, # Used in semantic search based on opensearch 

    # hybird-search debugger
    #hybrid_search_debugger = "semantic", #[semantic, lexical, None]
    
    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for search
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.51, .49], # [for semantic, for lexical], Semantic, Lexical search 결과에 대한 최종 반영 비율 정의
    reranker=False, # enable reranker with reranker model
    #reranker_endpoint_name=endpoint_name, # endpoint name for reranking model
    parent_document=True, # enable parent document
    
    # option for complex pdf consisting of text, table and image
    complex_doc=True,
    
    # option for async search
    async_mode=True,

    # option for output
    k=7, # 최종 Document 수 정의
    verbose=False,

)

In [ ]:
%%time
query = "자동조기 상환"
search_hybrid_result, tables, images = opensearch_hybrid_retriever.invoke(query)

In [ ]:
show_context_used(search_hybrid_result)